In [1]:
import geemap
import ee

In [2]:
# Initialiser GEE avec son compte
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
Map = geemap.Map(center=[12.5, -13.5], zoom=8)
Map.add_basemap('SATELLITE')

In [18]:
hydrobasins = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_7")
faleme_basin = ee.Filter.inList('HYBAS_ID', [1070678260, 1070632050 , 1070677870, 1070720030, 1070717290,1070717070,1070720020, 1070713070])

faleme = hydrobasins.filter(faleme_basin)

Map.addLayer(faleme, {}, "Falémé")

In [19]:
faleme_geometry = faleme.geometry()

In [21]:
# Ajouter une collection d'image GEE
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filter(
    ee.Filter.date('2016-01-01','2016-12-31')
).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',100)).filterBounds(faleme_geometry).median().clip(faleme_geometry)

# Paramètres de visualisation 
vis = {
    "min": 0.0,
    "max": 7000,
    "bands": ["B4", "B3", "B2"],
}
Map.addLayer(sentinel2, vis, 'Sentinel-2')
Map

Map(bottom=30594.0, center=[13.464421817388486, -11.513671875000002], controls=(WidgetControl(options=['positi…

In [ ]:
waterThreshold = 0
def waterfunction(image):
    ndvi = image.clip(faleme_geometry).normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.clip(faleme_geometry).normalizedDifference(['B3','B8']).rename('NDWI')
    water01 = ndwi.lt(waterThreshold)
    image = image.updateMask(water01).addBands(ndvi)
    
    area = ee.Image.pixelArea()
    waterArea = water01.multiply(area).rename('waterArea')
    image = image.addBands(waterArea)
    
    stats = waterArea.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = faleme_geometry,
        scale = 30,
        maxPixels = 1e13
        )
        
    return image.set(stats)
        
collection = sentinel2.map(waterfunction)
print("Sentinel 2 collection", collection)


In [17]:
mp = geemap.Map(center=[12.5, -13.5], zoom=8)
mp.add_basemap('SATELLITE')
mp.addLayer(faleme_geometry, {}, "Falémé")
mp.addLayer(collection, {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'water detected')
mp

Map(center=[12.5, -13.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [ ]:
# Appliquer un seuil au NDWI pour détecter l'eau
water_mask = ndwi.gt(0)

# Calculer les superficies d'eau en mètres carrés
water_area = water_mask.multiply(ee.Image.pixelArea())

# Réduire la région pour obtenir la superficie totale d'eau
total_water_area = water_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=water_mask.geometry(),
    scale=10,  # Résolution en mètres
    bestEffort=True  # Utiliser la meilleure résolution possible pour respecter la limite de pixels
)

# Obtenir la valeur totale de la superficie d'eau
total_water_area_value = total_water_area.get('nd').getInfo()

print(f"Superficie totale d'eau: {total_water_area_value} m²")

Superficie totale d'eau: 1701.186279296875 m²


In [ ]:
# Ajouter les surface d'eau à la carte
Map_area = geemap.Map(center=[12.5, -13.5], zoom=8)
Map_area.add_basemap('SATELLITE')
Map_area.addLayer(faleme_geometry, {}, "Falémé")
Map_area.addLayer(water_area, {'min': -1, 'max': 1, 'palette': ['blue']}, 'Water area')
Map_area

Map(center=[12.5, -13.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…